# Funções
Arquivo de funções de machine Learning para prever boas apostas de futebol para a Bet365 na temporada 2021/2022 da Premier League

In [49]:
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import re

In [66]:
# Carregue o conjunto de dados
db = pathlib.Path.cwd() / "../data/final_dataset.csv"
data = pd.read_csv(db)

# Filtrar os resultados antes de 31/06/2021
train_data = data[data['Date'] < '2021-06-30']

# Mantenha apenas as colunas "HomeTeam", "AwayTeam" e "FTR" (resultado)
train_data = train_data[["HomeTeam", "AwayTeam", "FTR"]]

# Verifique se há valores ausentes
train_data.isnull().sum()

# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
train_data_encoded = pd.get_dummies(train_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separe as features (variáveis de entrada) e o target (variável de saída)
X_train = train_data_encoded.drop("FTR", axis=1)
y_train = train_data_encoded["FTR"]

# Filtrar os resultados após 30/06/2021 para teste
test_data = data[data['Date'] >= '2021-06-30']

# Mantenha apenas as colunas "HomeTeam", "AwayTeam" e "FTR" (resultado)
test_data = test_data[["HomeTeam", "AwayTeam", "FTR"]]

# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_data_encoded = pd.get_dummies(test_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separe as features (variáveis de entrada) e o target (variável de saída)
X_test = test_data_encoded.drop("FTR", axis=1)
y_test = test_data_encoded["FTR"]

# Crie e ajuste o modelo de regressão logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Crie e ajuste o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Crie e ajuste o modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Crie e ajuste o modelo SVM
svm = SVC()
svm.fit(X_train, y_train)

# Crie e ajuste o modelo Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

GaussianNB()

In [67]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC

# Função para fazer uma sugestão de aposta com base no modelo treinado
def fazer_sugestao_aposta(jogo, modelo):
    # Criar DataFrame com as informações do jogo
    jogo_df = pd.DataFrame(jogo, index=[0])
    
    # Aplicar one-hot encoding nas variáveis categóricas
    jogo_encoded = pd.get_dummies(jogo_df, columns=["HomeTeam", "AwayTeam"], drop_first=True)
    
    # Preencher colunas ausentes no conjunto de dados do jogo
    missing_cols = set(X_train.columns) - set(jogo_encoded.columns)
    for col in missing_cols:
        jogo_encoded[col] = 0
    
    # Ordenar as colunas do conjunto de dados do jogo
    jogo_encoded = jogo_encoded[X_train.columns]
    
    # Fazer a previsão com base no modelo treinado
    predicao = modelo.predict(jogo_encoded)
    print("Predição:", predicao)
    
    # Mapear a previsão para uma sugestão de aposta
    if predicao[0] == 'H':
        sugestao = "Apostar no time da casa (H)"
    elif predicao[0] == 'A':
        sugestao = "Apostar no time visitante (A)"
    else:
        sugestao = "Apostar no empate (D)"
    
    return sugestao

# Carregar o modelo treinado
modelo = random_forest

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Aston Villa',
    "AwayTeam": 'Man United',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = fazer_sugestao_aposta(jogo, modelo)

print("Sugestão de aposta:", sugestao_aposta)

Predição: ['NH']
Sugestão de aposta: Apostar no empate (D)


In [68]:
# Faça a previsão no conjunto de teste
y_pred = logreg.predict(X_test)
# Avalie o modelo no conjunto de teste
accuracy = logreg.score(X_test, y_test)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.6447784810126582


In [69]:
# Faça a previsão no conjunto de teste
y_pred = decision_tree.predict(X_test)
# Avalie a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)


Acurácia do modelo: 0.6071993670886076


In [70]:
# Faça previsões no conjunto de teste
y_pred = random_forest.predict(X_test)
# Avalie a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.6107594936708861


In [71]:
# Faça previsões no conjunto de teste
y_pred = svm.predict(X_test)
# Avalie a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)


Acurácia do modelo: 0.6277689873417721


In [72]:
# Faça previsões no conjunto de teste
y_pred = naive_bayes.predict(X_test)
# Avalie a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)


Acurácia do modelo: 0.6257911392405063
